In [3]:

import pandas as pd
from logic.factory_manage.sql import load_data_from_sqlite

# CSVファイル読み込み（パスは適宜変更）
path = "/work/app/data/factory_manage/weight_data.db"
df = load_data_from_sqlite()
df["伝票日付"].max()

Timestamp('2025-06-18 00:00:00')

In [4]:
from utils.get_holydays import get_japanese_holidays
hol_max = df["伝票日付"].max()
hol_min = df["伝票日付"].min()
holiday = get_japanese_holidays(hol_min, hol_max)
holiday[:5]

['2024-05-03', '2024-05-04', '2024-05-05', '2024-05-06', '2024-07-15']

In [9]:
from logic.factory_manage.predict_model_ver2 import train_and_predict_with_holiday
import pandas as pd
from datetime import timedelta

roop_start_date = pd.to_datetime("2025-03-01")
roop_end_date = pd.to_datetime("2025-05-31")

results = []

for current_start in pd.date_range(start=roop_start_date, end=roop_end_date):
    start_dt = current_start
    end_dt = start_dt  # 同じ日に設定

    filtered_df = df[df['伝票日付'] < start_dt]

    if filtered_df.empty:
        print(f"Warning: No training data available before {start_dt.strftime('%Y-%m-%d')}, skipping.")
        continue

    print(f"date {filtered_df['伝票日付'].min()} to {filtered_df['伝票日付'].max()} with {len(filtered_df)} records")
    print(f"predicting for {start_dt.strftime('%Y-%m-%d')}")

    pred_df = train_and_predict_with_holiday(
        filtered_df,
        start_dt.strftime("%Y-%m-%d"),
        end_dt.strftime("%Y-%m-%d"),
        holiday
    )

    # 予測結果に予測対象日カラムを追加
    pred_df['予測対象日'] = start_dt.strftime('%Y-%m-%d')

    results.append(pred_df)

if results:
    all_results_df = pd.concat(results, ignore_index=True)
    all_results_df.to_csv("predictions_.csv", index=False)
    print("Prediction results saved to predictions_202505.csv")
else:
    print("No prediction results to save.")


date 2024-05-01 00:00:00 to 2025-02-28 00:00:00 with 42691 records
predicting for 2025-03-01
✅ R² = 0.999, MAE = 572 kg
date 2024-05-01 00:00:00 to 2025-03-01 00:00:00 with 42865 records
predicting for 2025-03-02
✅ R² = 0.999, MAE = 561 kg
date 2024-05-01 00:00:00 to 2025-03-02 00:00:00 with 42951 records
predicting for 2025-03-03
✅ R² = 0.999, MAE = 499 kg
date 2024-05-01 00:00:00 to 2025-03-03 00:00:00 with 43132 records
predicting for 2025-03-04
✅ R² = 0.999, MAE = 437 kg
date 2024-05-01 00:00:00 to 2025-03-04 00:00:00 with 43311 records
predicting for 2025-03-05
✅ R² = 0.999, MAE = 605 kg
date 2024-05-01 00:00:00 to 2025-03-05 00:00:00 with 43473 records
predicting for 2025-03-06
✅ R² = 0.999, MAE = 544 kg
date 2024-05-01 00:00:00 to 2025-03-06 00:00:00 with 43663 records
predicting for 2025-03-07
✅ R² = 0.999, MAE = 676 kg
date 2024-05-01 00:00:00 to 2025-03-07 00:00:00 with 43856 records
predicting for 2025-03-08
✅ R² = 0.999, MAE = 493 kg
date 2024-05-01 00:00:00 to 2025-03-08 0

/tmp/ipykernel_12851/3038692180.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_results_df = pd.concat(results, ignore_index=True)
